# Decsion Trees are based on three points named Entropy , information gain and giny impurity
# Ideal selection of nodes and predicting separate classes requires low entropy , high information gain , and low genie impurity

# Entropy is the measure of randomness in the data with the formula H(S) = - sum(p_i * log2(p_i))
# information gain computes the diffrence between the entropy after split and and average entropy after split(low entropy high information gain
# and high entropy low information gain) IG(S, A) = H(S) - sum(|S_v|/|S| * H(S_v))
 # where IG(S, A) is the information gain of attribute A on set S, H(S) is the entropy of set S, |S_v| is the number of examples in subset S_v that have value v for attribute A, and |S| is the total number of examples in set S.

# The summation is taken over all possible values of attribute A. The attribute that maximizes information gain is chosen as the splitting attribute at the current node.

# To build a decision tree, the data is recursively split based on the attributes that provide the most information gain until a stopping criterion is met, such as all examples in a node belonging to the same class or the tree reaches a certain depth.

# Information gain is a key concept in decision trees and is used to construct decision trees that are able to make accurate predictions for new, unseen data

In [117]:
# installing the dependencies

import pandas as pd
import numpy as np
from pprint import pprint
from collections import Counter


In [118]:
# importing the data set
df = pd.read_csv("zoo.csv",names=['Animal_name','air','Feathers','Eggs','Milk','Airborne','Aquatic','Predator','Toothed','Backbone'
,'Breathes','Venomous','Fins','Legs','Tail','Domestic','Catsize','Class_type'])




In [119]:
df.drop(index=0 ,axis= 0,inplace= True)


In [120]:
df.drop('Animal_name',axis=1,inplace= True)  # the animal name is dropped as it irrelevant

# designing the decision tree

# Train algorithm == build the tree
1> start at the top node and at each node select the best split based on the nformation given
2>Greedy search loop all over th features and over all thresholds
3> save the best split feature and split threshold at each node
4> build the tree recursively
5> apply stopping criteria
maxmum depth , mimun samples at each node , no more class disttibution(entropy becomes zero)
6> when we have a leaf node store the most common class



# Predict ==== Traverse the tree
1> Traverse the tree recursivly
2> at each node we look at the best split feature vetorx and go left or right depending on x[feature_idx] <= threshold>

3> when we reach the leaf node we return the store the most common class label

In [121]:
# vector y siginfies all the class labels  (entropy is  E=-Σ⋅p(x)⋅log_2⁡(p(x))
# p(x) = #x/n  (x is the count of instances) = n - number of occurences

def entropy(y):
    hist =np.bincount(y)
    ps = hist /len(y)

    entro = -np.sum([p * np.log2(p) for p in ps if p > 0])   # the condition as log2 doesnt work on -ve numbers
    return entro


class node:

    def __init__(self, feature = None, threshold = None , left = None, right = None,*,value = None):
        
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    
    def  is_leaf_node(self):  # a partuclar value based on target class has been assigned to the leaf node no further split
        return self.value is not None
    

class DecisionTree:

    def __init__(self, min_samples_split = 2 , max_depth = 100 , no_of_features = None):

        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.no_of_features = no_of_features
        self.root = None


    def fit(self,X,y):
        #grow tree   #safety feature

        self.no_of_features = X.shape[1] if not self.no_of_features else min(self.no_of_features,X.shape[1])
        self.root = self.grow_tree(X,y)

    def grow_tree(self,X,y,depth =0):

        n_sample ,n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if(depth >= self.max_depth
           or n_labels ==1
           or n_sample < self.min_samples_split):
            
            leaf_value = self.most_common_label(y)

            return node(value=leaf_value)

        feat_indexes = np.random(n_features,self.no_of_features, replace = False)

    def predict(self,X):
        #traverse the tree
        pass

    
    def most_common_label(self,y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common


In [129]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
classifier = DecisionTreeClassifier()

In [123]:
from sklearn.model_selection import train_test_split

X = df.drop("Class_type",axis= 1)
y = df[["Class_type"]]

In [124]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.3,random_state= 3)

In [125]:
classifier.fit(X_train,y_train)

DecisionTreeClassifier()

In [126]:
Y_pred = classifier.predict(X_test)

In [131]:
accuracy_score(y_test,Y_pred)



0.9354838709677419